## 심화통계 JOIN

In [25]:
# Import Libraries
#초기 설정및 시스템 라이브러리
import platform
import warnings

# 데이터 시각화 라이브러리
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
print(platform.system())
warnings.filterwarnings('ignore')

# 행,열,결과값 생략 없이 보기,세팅
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)
%matplotlib inline

# 시각화 OS별 한글폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 폰트 설정
elif platform.system() == 'Mac':
    plt.rcParams['font.family'] = 'AppleGothic'  # Mac 폰트 설정
    
print("="*60)
print("라이브러리 로드 완료!")
print("한글 폰트 설정 완료!")
print("="*60)

Windows
라이브러리 로드 완료!
한글 폰트 설정 완료!


In [26]:
# Load Dataset
s_master   = pd.read_csv("./data/stat/success_master.csv")              # success_master
f_master   = pd.read_csv("./data/stat/invested_founder_master.csv")     # invested_founder_master
s_profile  = pd.read_csv("./data/stat/startup_profile.csv")             # startup_profile

rel = pd.read_csv("./data/clean/clean_relationships_final.csv")         # clean_relationships_final
univ_rank = pd.read_csv("./data/stat/top100_univ.csv")                  # top100 university rankings
print("="*60)
print("데이터셋 로드 완료!")
print("="*60)

데이터셋 로드 완료!


In [27]:
print("s_master: ", s_master.shape)
print("f_master: ", f_master.shape)
print("s_profile: ", s_profile.shape)

s_master:  (483539, 58)
f_master:  (14747, 10)
s_profile:  (194151, 51)


In [28]:
print(s_master.columns)
print(f_master.columns)
print(s_profile.columns)

Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months', 'acquired_c_id', 'acquisition_id',
       'acquiring_c_id', 'acquir

In [29]:
print("[s_profile]", s_profile.duplicated().sum())
print("[s_master]",s_master.duplicated().sum())
print("[f_master]",f_master.duplicated().sum())

[s_profile] 0
[s_master] 0
[f_master] 0


In [30]:
# s_master grain 확인 → ✅ 완료
print("[s_master grain]", s_master[["objects_cfpr_id", "funding_round_id"]].duplicated().sum())

[s_master grain] 0


In [31]:
# f_master grain 확인 → 🚨 공동창업자 
print("[f_master grain]", f_master['rel_p_id'].duplicated().sum())

[f_master grain] 871


In [32]:
f_master.columns

Index(['rel_p_id', 'rel_cf_id', 'n_founding', 'is_past', 'sequence',
       'birthplace', 'degree_level', 'subject', 'institution', 'graduated_at'],
      dtype='object')

In [33]:
print(univ_rank.columns)
display(univ_rank.head())

Index(['rank_display', 'rank', 'tied_rank', 'university', 'city_inline',
       'city', 'country_or_region', 'location_raw'],
      dtype='object')


,rank_display,rank,tied_rank,university,city_inline,city,country_or_region,location_raw
0,1,1,False,Massachusetts Institute of Technology (MIT),Cambridge,Cambridge,United States,"Cambridge, United States"
1,2,2,False,Imperial College London,London,London,United Kingdom,"London, United Kingdom"
2,3,3,False,Stanford University,Stanford,Stanford,United States,"Stanford, United States"
3,4,4,False,University of Oxford,Oxford,Oxford,United Kingdom,"Oxford, United Kingdom"
4,5,5,False,Harvard University,Cambridge,Cambridge,United States,"Cambridge, United States"


#### 심화통계를 위한 조인

##### ◼ 스타트업별 창업자 명문대학 여부 플래그(is_univ_top100) 생성

In [34]:
import unicodedata
import re

# 학교명 정규화 함수
def norm_school(x: str) -> str:
    if pd.isna(x):
        return ""
    x = str(x).strip().lower()

    # 유니코드 정규화 (예: é, ü 등 처리 일관화)
    x = unicodedata.normalize("NFKC", x)

    # 괄호 내용 제거: "Harvard University (HBS)" -> "Harvard University"
    x = re.sub(r"\([^)]*\)", "", x)

    # &, / 등은 공백으로 치환
    x = re.sub(r"[&/]", " ", x)

    # 영숫자/공백 외 제거
    x = re.sub(r"[^a-z0-9\s]", " ", x)

    # 중복 공백 정리
    x = re.sub(r"\s+", " ", x).strip()
    return x

# top100 대학명 집합(set) 
qs_univ_set = set(univ_rank["university"].map(norm_school))
print("QS top100 unique normalized names:", len(qs_univ_set))

# 학교명 매핑
f_master["institution_norm"] = f_master["institution"].map(norm_school)

# top100 플래그 생성
f_master["is_univ_top100"] = f_master["institution_norm"].isin(qs_univ_set).astype(int)

# 결과 확인
print(f_master["is_univ_top100"].value_counts(dropna=False))

QS top100 unique normalized names: 100
is_univ_top100
0    12899
1     1848
Name: count, dtype: int64


##### ◼ 스타트업별 창업자 수(n_founders) 컬럼 생성

In [35]:
# 스타트업별 창업자 수(n_founders) 컬럼 생성
f_master['n_founders'] = (
    f_master
        .groupby('rel_cf_id')['rel_cf_id']
        .transform('size')
)
f_master["n_founders"].value_counts()

n_founders
2     5642
1     3880
3     3315
4     1304
5      395
6      102
8       48
7       42
10      10
9        9
Name: count, dtype: int64

##### ◼ 스타트업별 전공 STEM비율 플래그 생성

In [36]:
# f_master에 STEM 비율 플래그 생성
# STEM 여부 플래그 컬럼 생성: STEM = 1, notSTEM + NaN = 0
stem_subjects = [
    'Computer Science / Software',
    'AI / Data / Robotics',
    'Physics / Chemistry',
    'Biology / Life Science',
    'Medicine / Clinical Science',
    'Biotechnology / Pharmaceutical',
    'Mechanical / Industrial Engineering',
    'Electrical / Electronic Engineering'
]

f_master['is_stem'] = f_master['subject'].isin(stem_subjects).astype(int)

# 스타트업별 STEM 비율 → f_master 바로 추가
f_master['subject_stem'] = (
    f_master
        .groupby('rel_cf_id')['is_stem']
        .transform('mean').round(2)
)

# 확인
f_master['subject_stem'].describe()

count    14747.000000
mean         0.222660
std          0.341285
min          0.000000
25%          0.000000
50%          0.000000
75%          0.500000
max          1.000000
Name: subject_stem, dtype: float64

In [37]:
f_master.columns

Index(['rel_p_id', 'rel_cf_id', 'n_founding', 'is_past', 'sequence',
       'birthplace', 'degree_level', 'subject', 'institution', 'graduated_at',
       'institution_norm', 'is_univ_top100', 'n_founders', 'is_stem',
       'subject_stem'],
      dtype='object')

In [38]:
f_master["rel_p_id"].duplicated().sum()

np.int64(871)

##### ◼ 스타트업별 직무다양성(title_diversity) 컬럼 생성

In [39]:
# 직무다양성 컬럼 생성
rel['title_diversity'] = (
    rel.groupby('rel_cf_id')['cat_rel_title']
       .transform('nunique')
)

# 회사별 대표 1명 선택 (단계적 규칙 적용)
def select_rep_person(df):
    # 1) sequence 최소
    min_seq = df['sequence'].min()
    df1 = df[df['sequence'] == min_seq]

    # 2) is_past == 0 우선
    if (df1['is_past'] == 0).any():
        df2 = df1[df1['is_past'] == 0]
    else:
        df2 = df1

    # 3) relationship_id 최소
    return df2.loc[df2['relationship_id'].idxmin()]

rel_rep = (
    rel
    .groupby('rel_cf_id', as_index=False, group_keys=False)
    .apply(select_rep_person)
    .reset_index(drop=True)
)

use_cols = ["rel_cf_id", "title_diversity"]
rel_rep_diversity = rel_rep[use_cols].copy()
rel_rep_diversity.head()

,rel_cf_id,title_diversity
0,c:1,7
1,c:10,4
2,c:100,7
3,c:1000,1
4,c:10002,2


In [40]:
print("[rel_rep_diversity grain]", rel_rep_diversity["rel_cf_id"].duplicated().sum())

[rel_rep_diversity grain] 0


In [41]:
# f_master ← rel_rep_diversity "title_diversity"
f_master = (
    f_master
    .merge(
        rel_rep_diversity,
        on="rel_cf_id",
        how="left"
    )
)
f_master.columns

Index(['rel_p_id', 'rel_cf_id', 'n_founding', 'is_past', 'sequence',
       'birthplace', 'degree_level', 'subject', 'institution', 'graduated_at',
       'institution_norm', 'is_univ_top100', 'n_founders', 'is_stem',
       'subject_stem', 'title_diversity'],
      dtype='object')

In [42]:
f_master[["rel_p_id", "rel_cf_id"]].duplicated().sum()

np.int64(0)

##### ◼ 스타트업별 창업자 출생지(birthplace) 정규화

In [43]:
f_master['birthplace'].unique()

array([nan, 'United States', 'Japan', 'Turkey', 'sg', 'Portugal', 'split',
       'Ireland', 'New Zealand', 'France', 'Italy', 'Israel', 'Germany',
       'Russia', 'Australia', 'India', 'hong kong', 'United Kingdom',
       'taiwan', 'nairobi', 'Greece', 'Argentina', 'Denmark', 'Singapore',
       'Canada', 'antwerp', 'Belgium', 'Iran', 'South Africa', 'bonn',
       'essen', 'Bangladesh', 'novi sad', 'Chile', 'Sweden', 'tel-aviv',
       'Costa Rica', 'Poland', 'Czech Republic', 'Hungary', 'Brazil',
       'Switzerland', 'Spain', 'a', 'Ukraine', 'Austria', 'lodo',
       'exeter', 'Colombia', 'China', 'zhytomyr', 'Korea', 'ravenna',
       'cz', 'reykjavik', 'europe', 'Norway', 'e', 'c', 'augsburg',
       'Netherlands', 'vernon, bc', 'Mexico', 'Finland', 'slough', 'kent',
       'postojna', 'egypt', 'Jamaica', 'toulon', 'tunis', 'honiton',
       'ashton', 'aachen', 'Pakistan', 'brovst', 'zimbabwe', 'naples',
       'breda', 'brugge', 'pei', 'goulburn, nsw'], dtype=object)

In [44]:
# 텍스트 정규화
def normalize_text(x):
    if pd.isna(x):
        return np.nan
    x = x.lower()
    x = re.sub(r'\(.*?\)', '', x)      # 괄호 제거
    x = re.sub(r'[^a-z\s]', ' ', x)    # 특수문자 제거
    x = re.sub(r'\s+', ' ', x).strip() # 공백 정리
    return x

f_master['birthplace_norm'] = f_master['birthplace'].apply(normalize_text)

# 국가명 매핑
country_alias = {
    'usa': 'United States',
    'us': 'United States',
    'united states': 'United States',
    'uk': 'United Kingdom',
    'united kingdom': 'United Kingdom',
    'uae': 'United Arab Emirates',
    'sg': 'Singapore',
    'nz': 'New Zealand',
    'cz': 'Czech Republic',
    'korea': 'South Korea',
    'south korea': 'South Korea',
    'taiwan': 'Taiwan',
    'hong kong': 'Hong Kong',
    'peoples republic of china': 'China',
    'china': 'China',
}

# 도시 -> 국가 매핑
city_to_country = {
    # US
    'san antonio': 'United States',
    'burbank': 'United States',
    'redwood city': 'United States',
    'baton rouge': 'United States',
    'youngstown': 'United States',
    'fullerton': 'United States',
    'hollywood': 'United States',
    'long beach': 'United States',

    # UK
    'london': 'United Kingdom',
    'durham': 'United Kingdom',
    'exeter': 'United Kingdom',
    'preston': 'United Kingdom',
    'slough': 'United Kingdom',
    'kent': 'United Kingdom',
    'epsom': 'United Kingdom',

    # Germany
    'nuremberg': 'Germany',
    'hannover': 'Germany',
    'bonn': 'Germany',
    'essen': 'Germany',
    'bochum': 'Germany',
    'augsburg': 'Germany',
    'aachen': 'Germany',
    'siegen': 'Germany',

    # France
    'angers': 'France',
    'poitiers': 'France',
    'toulon': 'France',

    # Italy
    'verona': 'Italy',
    'naples': 'Italy',
    'empoli': 'Italy',
    'fiesole': 'Italy',
    'ravenna': 'Italy',

    # Spain
    'oviedo': 'Spain',

    # Japan
    'tokyo': 'Japan',

    # India
    'agra': 'India',
    'jodhpur': 'India',
    'kolkatta': 'India',
    'guntur': 'India',
    'theni': 'India',
    'sirsā': 'India',

    # Netherlands / Belgium
    'breda': 'Netherlands',
    'antwerp': 'Belgium',
    'brugge': 'Belgium',
}

# 최종 매핑 함수
def map_to_country(x):
    if pd.isna(x):
        return np.nan

    # 1) 국가 alias 우선
    if x in country_alias:
        return country_alias[x]

    # 2) 도시 매핑
    if x in city_to_country:
        return city_to_country[x]

    # 3) 명확한 국가명 패턴
    if len(x.split()) <= 3 and x.title() in country_alias.values():
        return x.title()

    # 4) 그 외 → NaN
    return np.nan

f_master['birthplace_country'] = (
    f_master['birthplace_norm']
    .apply(map_to_country)
)
f_master["birthplace_country"].unique()

array([nan, 'United States', 'Singapore', 'New Zealand', 'Hong Kong',
       'United Kingdom', 'Taiwan', 'Belgium', 'Germany', 'Czech Republic',
       'China', 'South Korea', 'Italy', 'France', 'Netherlands'],
      dtype=object)

##### ◼ 베이스 국가 설정

In [45]:
# 미국 출생 여부 플래그 생성
# 1. 미국 출생 여부 플래그
f_master['is_us_born'] = (
    f_master['birthplace_country'] == 'United States'
).astype(int)

# 2. 스타트업별 미국 출생 창업자 비율 → f_master에 바로 추가
f_master['us_born_ratio'] = (
    f_master
        .groupby('rel_cf_id')['is_us_born']
        .transform('mean')
)

#### f_master 집계

In [46]:
f_master.columns

Index(['rel_p_id', 'rel_cf_id', 'n_founding', 'is_past', 'sequence',
       'birthplace', 'degree_level', 'subject', 'institution', 'graduated_at',
       'institution_norm', 'is_univ_top100', 'n_founders', 'is_stem',
       'subject_stem', 'title_diversity', 'birthplace_norm',
       'birthplace_country', 'is_us_born', 'us_born_ratio'],
      dtype='object')

In [47]:
# =========================================
# STEP 0. 필요한 컬럼 가정
# rel_cf_id, rel_p_id, sequence, is_past
# n_founding, degree_level, subject_stem
# n_founders, is_univ_top100, us_born_ratio
# =========================================


# =========================================
# STEP 1. 스타트업별 대표 창업자 선택
# - sequence 가장 작은 창업자 먼저 선택
# - 그중에서 is_past == 0 인 경우만 유지
# =========================================

rep_founder = (
    f_master
    .sort_values(['rel_cf_id', 'sequence'])
    .groupby('rel_cf_id', as_index=False)
    .head(1)
    .query('is_past == 0')
    [['rel_cf_id', 'rel_p_id']]
)


# =========================================
# STEP 2. 창업자 특성 집계 (startup-level)
# - 모든 창업자 기준 집계
# =========================================

founder_agg = (
    f_master
    .groupby('rel_cf_id', as_index=False)
    .agg(
        n_founding_max   = ('n_founding', 'max'),
        degree_max      = ('degree_level', 'max'),
        stem_ratio      = ('subject_stem', 'mean'),
        co_founders     = ('n_founders', 'mean'),
        top_university  = ('is_univ_top100', 'mean'),
        us_born_ratio   = ('us_born_ratio', 'mean'),
        title_diversity = ('title_diversity', 'mean')
    )
)


# =========================================
# STEP 3. 대표 창업자 + 집계 결과 결합
# - 최종 grain: rel_cf_id 1행
# =========================================

f_master_by_founder = (
    rep_founder
    .merge(founder_agg, on='rel_cf_id', how='left')
)


# =========================================
# STEP 4. 검증 (필수)
# =========================================

print("shape:", f_master_by_founder.shape)
print("rel_cf_id unique:", f_master_by_founder['rel_cf_id'].is_unique)
f_master_by_founder.head()

shape: (6516, 9)
rel_cf_id unique: True


,rel_cf_id,rel_p_id,n_founding_max,degree_max,stem_ratio,co_founders,top_university,us_born_ratio,title_diversity
0,c:1,p:59304,2,2.0,1.0,2.0,0.0,0.0,7.0
1,c:10014,p:16925,3,NaN,0.0,1.0,0.0,0.0,4.0
2,c:10015,p:16930,3,0.0,1.0,1.0,1.0,0.0,9.0
3,c:10018,p:17332,1,NaN,0.0,1.0,0.0,0.0,7.0
4,c:100189,p:23613,1,NaN,0.0,1.0,0.0,0.0,2.0


In [48]:
print("rel_cf_id unique:", f_master_by_founder['rel_cf_id'].is_unique)

rel_cf_id unique: True


<span style="font-size: 15px;">
◼ 스타트업별 금액정보 공개 플래그 생성<br>
</span>

In [49]:
s_master.columns

Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months', 'acquired_c_id', 'acquisition_id',
       'acquiring_c_id', 'acquir

In [50]:
s_master[["objects_cfpr_id", "funding_round_id"]].duplicated().sum()

np.int64(0)

In [51]:
# 금액 공개율 컬럼 생성
open_rate_df = (
    s_master
    .groupby(['objects_cfpr_id','funding_round_id'])['is_fr_raised_private']
    .mean()
    .rename('private_rate')
    .reset_index()
)

# 공개율 = 1 - 비공개율
open_rate_df['open_rate'] = 1 - open_rate_df['private_rate']
open_rate_df.columns

Index(['objects_cfpr_id', 'funding_round_id', 'private_rate', 'open_rate'], dtype='object')

In [52]:
open_rate_df[["objects_cfpr_id", "funding_round_id"]].duplicated().sum()

np.int64(0)

In [53]:
# s_master에 공개율 컬럼 병합
use_cols = [
    "open_rate", "private_rate"
]
s_master = (
    s_master
    .merge(
        open_rate_df[use_cols + ["objects_cfpr_id", "funding_round_id"]],
        on=["objects_cfpr_id", "funding_round_id"],
        how="left"
    )
)
print(s_master.columns)
print(s_master[["objects_cfpr_id","funding_round_id"]].duplicated().sum())

Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months', 'acquired_c_id', 'acquisition_id',
       'acquiring_c_id', 'acquir

#### s_master 집계
<span style="font-size: 15px;">
success_flag, round_tempo_months<br>
</span>

In [54]:
s_master.columns

Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months', 'acquired_c_id', 'acquisition_id',
       'acquiring_c_id', 'acquir

In [55]:
# objects_cfpr_id & funding_round_id 기준 집계
# funding_round_id를 결측 허용 정수로 정리
s_master["funding_round_id"] = pd.to_numeric(
    s_master["funding_round_id"], errors="coerce"
).astype("Int64")

s_master_by_company = (
    s_master
    .groupby(["objects_cfpr_id", "funding_round_id"], as_index=False)
    .agg(
        success_flag=("success_flag", "max"),
        round_tempo_months=("round_tempo_months", "first"),
        open_rate=("open_rate", "first"),
    )
)
print(s_master_by_company.shape)
print(s_master_by_company.duplicated().sum())
s_master_by_company.head()

(52626, 5)
0


,objects_cfpr_id,funding_round_id,success_flag,round_tempo_months,open_rate
0,c:1,888,1,NaN,1.0
1,c:1,889,1,15.0,1.0
2,c:1,2312,1,17.0,1.0
3,c:1001,1644,1,NaN,1.0
4,c:10014,6682,1,NaN,0.0


In [57]:
print("objects_cfpr_id unique:", s_master_by_company[['objects_cfpr_id', 'funding_round_id']].duplicated().sum())

objects_cfpr_id unique: 0


#### 최종조인
<span style="font-size: 15px;">
◼ 심화 통계를 위한 최종 조인<br>
</span>

In [58]:
# 심화통계를 위한 조인
# s_profile ← f_master ← s_master
sf_master = (
    s_profile
    # 1) s_profile ← f_master_by_founder
    .merge(
        f_master_by_founder,
        left_on="objects_cfpr_id",
        right_on="rel_cf_id",
        how="left"
    )
    # 2) f_master_by_founder ← s_master_by_company
    .merge(
        s_master_by_company,
        on=["objects_cfpr_id"],
        how="left"
    )
)

print("s_profile shape:", s_profile.shape)
print("f_master shape:", f_master.shape)
print("sf_master shape:", sf_master.shape)

s_profile shape: (194151, 51)
f_master shape: (14747, 20)
sf_master shape: (215055, 64)


<span style="font-size: 15px;">
◼ 1행 1스타트업 (중복제거) 확인 → ✅이상없음<br>
</span>

In [59]:
# 중복데이터 확인
print(sf_master.duplicated().sum())
print("objects_cfpr_id unique:", sf_master[["objects_cfpr_id", "funding_round_id"]].duplicated().sum())

0
objects_cfpr_id unique: 0


In [60]:
sf_master.to_csv("./data/stat/sf_master.csv", encoding="utf-8", index=False)
print("="*60)
print("csv 추출 완료!")
print("="*60)

csv 추출 완료!


In [61]:
sf_master.columns

Index(['objects_cfpr_id', 'entity_type', 'parent_c_id', 'normalized_name',
       'category_code', 'status', 'founded_at', 'closed_at', 'description',
       'overview', 'tag_list', 'country_code', 'state_code', 'city', 'region',
       'first_investment_at', 'last_investment_at', 'investment_rounds',
       'invested_companies', 'first_funding_at', 'last_funding_at',
       'funding_rounds', 'funding_total_usd', 'first_milestone_at',
       'last_milestone_at', 'milestones', 'relationships',
       'is_obj_parent_id_missing', 'is_obj_category_missing',
       'is_obj_founded_missing', 'is_obj_closed_missing',
       'is_obj_overview_missing', 'is_obj_state_missing',
       'is_obj_inv_rounds_missing', 'is_obj_inv_comp_missing',
       'cat_obj_status', 'obj_overview_fixed', 'cat_obj_overview',
       'obj_region_fixed', 'cat_obj_region', 'obj_state_filled',
       'obj_city_fixed', 'obj_category_filled',
       'is_obj_funding_total_usd_private', 'is_obj_funding_rounds_private',
     